In [0]:
#### python_ncut_lib.py
# Copyright (C) 2010 R. Cameron Craddock (cameron.craddock@gmail.com)
#
# This script is a part of the pyClusterROI python toolbox for the spatially
# constrained clustering of fMRI data. It provides the library functions for
# performing normalized cut clustering according to:
#
# Stella Yu and Jianbo Shi, "Understanding Popout through Repulsion," Computer
# Vision and Pattern Recognition, December, 2001.  
#
# Shi, J., & Malik, J. (2000).  Normalized cuts and image segmentation. IEEE
# Transactions on Pattern Analysis and Machine Intelligence, 22(8), 888-905.
# doi: 10.1109/34.868688.
#
# Yu, S. X., & Shi, J. (2003). Multiclass spectral clustering. Proceedings Ninth
# IEEE International Conference on Computer Vision, (1), 313-319 vol.1. Ieee.
# doi: 10.1109/ICCV.2003.1238361.
#
# This code is a port of the NcutClustering_7 matlab toolbox available here:
# http://www.cis.upenn.edu/~jshi/software/
#
# For more information refer to:
#
# Craddock, R. C.; James, G. A.; Holtzheimer, P. E.; Hu, X. P. & Mayberg, H. S.
# A whole brain fMRI atlas generated via spatially constrained spectral
# clustering Human Brain Mapping, 2012, 33, 1914-1928 doi: 10.1002/hbm.21333.
#
# ARTICLE{Craddock2012,
#   author = {Craddock, R C and James, G A and Holtzheimer, P E and Hu, X P and
#   Mayberg, H S},
#   title = {{A whole brain fMRI atlas generated via spatially constrained
#   spectral clustering}},
#   journal = {Human Brain Mapping},
#   year = {2012},
#   volume = {33},
#   pages = {1914--1928},
#   number = {8},
#   address = {Department of Neuroscience, Baylor College of Medicine, Houston,
#       TX, United States},
#   pmid = {21769991},
# } 
#
# Documentation, updated source code and other information can be found at the
# NITRC web page: http://www.nitrc.org/projects/cluster_roi/ and on github at
# https://github.com/ccraddock/cluster_roi
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
# 
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.
####

# this scripts requires NumPy (numpy.scipy.org) and SciPy (www.scipy.org) to be
# installed in a directory that is accessible through PythonPath 
import sys
from numpy import array, reshape, shape, matrix, ones, zeros, sqrt
from numpy import argsort, sign, kron, multiply, divide, abs, asarray
from numpy.random import rand
from scipy.sparse import csc_matrix, spdiags
from scipy.sparse.linalg import eigsh
from scipy.linalg import norm, svd, LinAlgError
import numpy as np

# exception hander for singular value decomposition
class SVDError(Exception):
    def __init__(self,value):
        self.value=value
    def __str__(self):
        return repr(self.value)


# (eigen_val, eigen_vec) = ncut( W, nbEigenValues ):
#
# This function performs the first step of normalized cut spectral clustering.
# The normalized LaPlacian is calculated on the similarity matrix W, and top
# nbEigenValues eigenvectors are calculated. The number of eigenvectors
# corresponds to the maximum number of classes (K) that will be produced by the
# clustering algorithm. 
#
#    W:             symmetric #feature x #feature sparse matrix representing the
#                   similarity between voxels, traditionally this matrix should
#                   be positive semidefinite, but regularization is employed to
#                   allow negative matrix entries (Yu 2001)
#    nvEigenValues: number of eigenvectors that should be calculated, this
#                   determines the maximum number of clusters (K) that can be
#                   derived from the
#    result
#    eigen_val:     (output) eigenvalues from the eigen decomposition of the
#                   LaPlacian of W
#    eigen_vec:     (output) eigenvectors from the eign decomposition of the
#                   LaPlacian of W
#
def ncut( W, nbEigenValues ):
    # parameters
    offset=.5
    maxiterations=100
    eigsErrorTolerence=1e-6
    eps=2.2204e-16

    m=shape(W)[1]

    # make sure that W is symmetric, this is a computationally expensive
    # operation, only use for debugging
    # if (W-W.transpose()).sum() != 0:
    #    print "W should be symmetric!"
    #    exit(0)

    # degrees and regularization
    # S Yu Understanding Popout through Repulsion CVPR 2001
    # Allows negative values as well as improves invertability of d for small
    # numbers i bet that this is what improves the stability of the eigen
    d=abs(W).sum(0)
    dr=0.5*(d-W.sum(0))
    d=d+offset*2
    dr=dr+offset

    # calculation of the normalized LaPlacian
    W=W+spdiags(dr,[0],m,m,"csc")
    Dinvsqrt=spdiags((1.0/sqrt(d+eps)),[0],m,m,"csc")
    P=Dinvsqrt*(W*Dinvsqrt);

    # perform the eigen decomposition
    eigen_val,eigen_vec=eigsh(P,nbEigenValues,maxiter=maxiterations,\
        tol=eigsErrorTolerence,which='LA')

    # sort the eigen_vals so that the first
    # is the largest
    i=argsort(-eigen_val)
    eigen_val=eigen_val[i]
    eigen_vec=eigen_vec[:,i]

    # normalize the returned eigenvectors
    eigen_vec=Dinvsqrt*matrix(eigen_vec)
    norm_ones=norm(ones((m,1)))
    for i in range(0,shape(eigen_vec)[1]):
        eigen_vec[:,i]=(eigen_vec[:,i] / norm(eigen_vec[:,i]))*norm_ones
        if eigen_vec[0,i] != 0:
            eigen_vec[:,i] = -1 * eigen_vec[:,i] * sign( eigen_vec[0,i] )

    return(eigen_val, eigen_vec)

# eigenvec_discrete=discretisation( eigen_vec ):
#
# This function performs the second step of normalized cut clustering which
# assigns features to clusters based on the eigen vectors from the LaPlacian of
# a similarity matrix. There are a few different ways to perform this task. Shi
# and Malik (2000) iteratively bisect the features based on the positive and
# negative loadings of the eigenvectors. Ng, Jordan and Weiss (2001) proposed to
# perform K-means clustering on the rows of the eigenvectors. The method
# implemented here was proposed by Yu and Shi (2003) and it finds a discrete
# solution by iteratively rotating a binaised set of vectors until they are
# maximally similar to the the eigenvectors (for more information, the full
# citation is at the top of this file). An advantage of this method over K-means
# is that it is _more_ deterministic, i.e. you should get very similar results
# every time you run the algorithm on the same data.
#
# The number of clusters that the features are clustered into is determined by
# the number of eignevectors (number of columns) in the input array eigen_vec. A
# caveat of this method, is that number of resulting clusters is bound by the
# number of eignevectors, but it may contain less.
#
#    eigen_vec:          Eigenvectors of the normalized LaPlacian calculated
#                        from the similarity matrix for the corresponding
#                        clustering problem
#    eigen_vec_discrete: (output) discretised eigenvectors, i.e. vectors of 0
#                        and 1 which indicate whether or not a feature belongs
#                        to the cluster defined by the eigen vector.  I.E. a one
#                        in the 10th row of the 4th eigenvector (column) means
#                        that feature 10 belongs to cluster #4.
# 
def discretisation( eigen_vec ):
    eps=2.2204e-16

    # normalize the eigenvectors
    [n,k]=shape(eigen_vec)
    vm=kron(ones((1,k)),sqrt(multiply(eigen_vec,eigen_vec).sum(1)))
    eigen_vec=divide(eigen_vec,vm)

    svd_restarts=0
    exitLoop=0

    ### if there is an exception we try to randomize and rerun SVD again
        ### do this 30 times
    while (svd_restarts < 30) and (exitLoop==0):

        # initialize algorithm with a random ordering of eigenvectors
        c=zeros((n,1))
        R=matrix(zeros((k,k)))
        R[:,0]=eigen_vec[int(rand(1)*(n-1)),:].transpose()

        for j in range(1,k):
            c=c+abs(eigen_vec*R[:,j-1])
            R[:,j]=eigen_vec[c.argmin(),:].transpose()


        lastObjectiveValue=0
        nbIterationsDiscretisation=0
        nbIterationsDiscretisationMax=20

        # iteratively rotate the discretised eigenvectors until they
        # are maximally similar to the input eignevectors, this 
        # converges when the differences between the current solution
        # and the previous solution differs by less than eps or we
        # we have reached the maximum number of itarations
        while exitLoop == 0:
            nbIterationsDiscretisation = nbIterationsDiscretisation + 1

            # rotate the original eigen_vectors
            tDiscrete=eigen_vec*R

            # discretise the result by setting the max of each row=1 and
            # other values to 0
            j=reshape(asarray(tDiscrete.argmax(1)),n)
            eigenvec_discrete=csc_matrix((ones(len(j)),(range(0,n), \
                array(j))),shape=(n,k))

            # calculate a rotation to bring the discrete eigenvectors cluster to
            # the original eigenvectors
            tSVD=eigenvec_discrete.transpose()*eigen_vec
            # catch a SVD convergence error and restart
            try:
                U, S, Vh = svd(tSVD)
            except LinAlgError:
                # catch exception and go back to the beginning of the loop
                print >> sys.stderr, \
                    "SVD did not converge, randomizing and trying again"
                break

            # test for convergence
            NcutValue=2*(n-S.sum())
            if((abs(NcutValue-lastObjectiveValue) < eps ) or \
                      ( nbIterationsDiscretisation > \
                        nbIterationsDiscretisationMax )):
                exitLoop=1
            else:
                # otherwise calculate rotation and continue
                lastObjectiveValue=NcutValue
                R=matrix(Vh).transpose()*matrix(U).transpose()

    if exitLoop == 0:
        raise SVDError("SVD did not converge after 30 retries")
    else:
        return(eigenvec_discrete)

    
class ClusteringNormalizedCuts:
    
    def __init__(self, n_clusters=4):
        self.__n_clusters = n_clusters
    
    def fit(self, X):
        self.__eigen_val, self.__eigen_vec = ncut(X, self.__n_clusters)
        res = discretisation(self.__eigen_vec).toarray()
        self.labels_ = np.zeros(X.shape[0])
        for i in range(res.shape[1]):
            self.labels_[res[:, i] == 1] = i
            
            
    def fit_predict(self, X):
        self.fit(X)
        return self.labels_

In [0]:


import numpy as np
import matplotlib.pyplot as plt 
from itertools import combinations
import pandas as pd
from  scipy.spatial.distance import cdist
from functools import partial
import multiprocessing as mlp

class KMeans:
    """
        K-Means clustering.
        
        Parameters
        ----------
        n_cluters : int, default=4
            The number of clusters to form as well as the number of centroids to generate.
            
        init : {'k-means++', 'random'}, default='k-means++'
            Method for initialization, defaults to 'k-means++':
            
            'k-means++' : selects initial cluster centers for k-mean 
            clustering in a smart way to speed up convergence.
            
            'random': choose k observations (rows) at random from data for
            the initial centroids.
 
        max_iter : int, default=100
            Maximum number of iterations of the k-means algorithm for a single run.
                 
        dist : str, default='euclidean'
            The distance metric to use.
        
        Attributes
        ----------
        cluster_centers_ : ndarray of shape (n_clusters, n_features)
            Coordiantes of cluster center.
        
        labels_ : ndarray of shape (n_samples, )
            Labels of each point.
        
        inertia_ : float
            Sum of squared distances of samples to their closest cluster center.
            
        n_iter_ : int
            Number of iterations run.   
    """
    
    
    def __init__(self, n_cluters=4, init='k-means++', max_iter=100, dist="euclidean"):
        """
            Initialize self.
        """
        self.__n_cluters = n_cluters
        self.__init = init
        self.__max_iter = max_iter
        self.__dist = dist

    def __initialisation(self, X):
        if self.__init == 'k-means++':
            self.cluster_centers_ = X[np.random.randint(0, X.shape[0], 1), :]
            for i in range(self.__n_cluters-1):
                dist = np.min(cdist(X, self.cluster_centers_, self.__dist), axis=1)
                self.cluster_centers_ = np.concatenate([self.cluster_centers_, 
                                                       X[np.argmax(dist), :].reshape(1,X.shape[1])], 
                                                       axis=0)
        else:
            self.cluster_centers = X[np.random.randint(0, X.shape[0], 3), :]

    def __affecte_cluster(self, X):
        dist = cdist(X, self.cluster_centers_, self.__dist)
        return np.argmin(dist, axis=1), np.min(dist, axis=1)

    def __nouveaux_centroides(self, X):
        return np.array([list((X[self.labels_ == i]).mean(axis=0)) for i in np.unique(self.labels_)])

    def __inertie_globale(self, cluster_dist):
        return np.sum([np.power(cluster_dist[self.labels_ == i], 2).sum() for i in np.unique(self.labels_)])


    def fit(self, X):
        """
            Compute k-means clustering.
            
            Parameters
            ----------
            X : array, shape=(n_samples, n_features)
                Training instances to cluster.
            
            Returns 
            -------
            self
                Fitted estimator.
        """
        self.__initialisation(X)
        self.inertia_ = float('inf')
        self.n_iter_ = 0
        for i in range(self.__max_iter):
            self.n_iter_ += 1
            if self.n_iter_ != 1:
                self.cluster_centers_ = self.__nouveaux_centroides(X)
            self.labels_, cluster_dist = self.__affecte_cluster(X)
            new_inertia = self.__inertie_globale(cluster_dist)
            if new_inertia == self.inertia_:
                self.inertia_ = new_inertia
                break
                
    def fit_predict(self, X):
        """
            Compute cluster centers and predict cluster index for each sample.
            
            Parameters
            ----------
            X : array, shape=(n_samples, n_features)
            
            Returns
            -------
            labels : array, shape [n_samples,]
                Index of the cluster each sample belongs to.
            
        """
        self.fit(X)
        return self.labels_

    
    
def average_confidence(CM, labels):
    """
        Compute the Average Confidence of assignment of the objects to its clusters.
         
        Parameters
        ----------
        CM : ndarray of shape(n_samples, n_samples)
            Co-association matrix.
         
        labels : ndarray of shape(n_samples, )
            Labels of clustering.
         
        Return
        ------
        float
            Average confidence of assignment of the objects to its clusters.
    """
    # compute the degree of confidence of assigning an object xi to its cluster Cp.
    n_cluster = np.unique(labels).size
    conf = np.zeros(labels.size)
    for xi in range(labels.size):
        xi_lab = labels[xi]
        smc = CM[xi, (labels == xi_lab) & (np.arange(labels.size) != xi)]
        if smc.size == 0 :
            smc = 0
        else: 
            smc = smc.mean()
        smac = []
        for j in range(n_cluster):
            if j != labels[xi]:
                smac.append(CM[xi, labels == j].mean())
        if len(smac) == 0:
            conf[xi] = smc
        else:
            conf[xi] = (smc - np.max(smac))
    return conf.mean()


def average_neighborhood_confidence(CM, labels, m=3):
    """
        Compute the Average Neighborhood Confidence of assigning the objects to its clusters.
  
        Parameters
        ----------
        CM : ndarray of shape(n_samples, n_samples)
            Co-association matrix.
         
        labels : ndarray of shape(n_samples, )
            Labels of clustering.
            
        m : int, default=3
            Number of neighbors.
         
        Return
        ------
         float
             Average  neighborhood confidence of assignment of the objects to its clusters.   
    """
    n_cluster = np.unique(labels).size
    conf = np.zeros(labels.size)
    if m == 0:
        return conf 
    for xi in range(labels.size):
        xi_lab = labels[xi]
        smc = CM[xi, (labels == xi_lab) & (np.arange(labels.size) != xi)]
        if smc.size == 0 :
            smc = 0
        elif smc.size >= m:
            smc = np.sort(smc)[::-1][:m].mean()
        else:
            smc = smc.mean()
        smac = []
        for j in range(n_cluster):
            if j != labels[xi]:
                smac_ = CM[xi, labels == j]
                if smac_.size > 0:
                    if smac_.size >= m:
                        smac_ = np.sort(smac_)[::-1][:m]
                    smac.append(smac_.mean())
        if len(smac) == 0:
            conf[xi] = smc
        else:
            conf[xi] = (smc - np.max(smac))
    return conf.mean()


def average_dynamique_neighborhood_confidence(CM, labels, alpha=0.5):
    """
        Compute the Average Dynamic Neighborhood Confidence of assigning the objects to its clusters.
        
        Parameters
        ----------
        CM : ndarray of shape(n_samples, n_samples)
            Co-association matrix.
         
        labels : ndarray of shape(n_samples, )
            Labels of clustering.
            
        alpha : float, default=0.5
            Use to calculate the number of neighbors dynamically.
         
        Return
        ------
         float
             Average dynamic neighborhood confidence of assigning the objects to its clusters.
    """
    n_cluster = np.unique(labels).size
    conf = np.zeros(labels.size)
    m = 1
    for xi in range(labels.size):
        xi_lab = labels[xi]
        smc = CM[xi, (labels == xi_lab) & (np.arange(labels.size) != xi)]
        m = int(np.floor(alpha * smc.sum()))
        if m == 0:
            m = 1
        if smc.size == 0 :
            smc = 0
        elif smc.size >= m:
            smc = np.sort(smc)[::-1][:m].mean()
        else:
            smc = smc.mean()
        smac = []
        for j in range(n_cluster):
            if j != labels[xi]:
                smac_ = CM[xi, labels == j]
                if smac_.size > 0:
                    if smac_.size >= m:
                        smac_ = np.sort(smac_)[::-1][:m]
                    smac.append(smac_.mean())
        if len(smac) == 0:
            conf[xi] = smc
        else:
            conf[xi] = (smc - np.max(smac))
    return conf.mean()


def vat(CM):
    """
        Compute the matrix for Visual Assessment of cluster Tendency.
        
        Parameters
        ----------
        CM : ndarray of shape(n_samlpes, n_samples)
            Co-association matrix, dissimilarity data.
        
        Return
        ------
        CM_VAT : ndarray of shape(n_samples, n_samples)
            Reorder dissimilarity data.
        
        Q : ndarray of shape (n_samples, )
            Reorder indexs of CM.
    """
    J = np.zeros((CM.shape), dtype=bool)
    I = np.ones((CM.shape), dtype=bool)
    Q = []
    cm_mask = None
    for t in range(CM.shape[0]):
        if t == 0:
            cm_mask = np.ma.masked_array(CM, mask = (I&J))
        else:
            cm_mask = np.ma.masked_array(CM, mask = (I|J))    
        i, j = np.unravel_index(cm_mask.argmin(), CM.shape)
        Q.append(j)
        J[:, j] = True
        I[j, :] = False
    cm_vat = CM.copy()
    for i in range(CM.shape[0]):
        for j in range(CM.shape[0]):
            cm_vat[i, j] = CM[Q[i], Q[j]]
            cm_vat[j, i] = CM[Q[i], Q[j]]
    return cm_vat, np.array(Q)


def gen_base_partition(X, k, It):
    return KMeans(k, max_iter=It).fit_predict(X)


def gen_base_partition_by_kmeans(X, M=10, It=4, Ktype='Random'):
    """
        Generate base partition by k-means
        
        Parameters
        ----------
        X : ndarray of shape (n_samples, n_features)
            data set.
        
        M : int default=10
            The number of base partitions
        
        It : int, default=10
            The number of iterations for k-means
        
        Ktype : {'Fixed', 'Random'}, default=Random
            The type to generate base partitions, 'Fixed' : k = sqrt(N)
        
        Return
        ------
        PI : ndarray of shape(n_sample, n_partitions)
            Base partitions, one column is a partition
    """
    N = X.shape[0]
        
    PI = []
    CM =  CM = np.zeros((X.shape[0], X.shape[0]))
    K = 0
    if Ktype == 'Fixed':
        k = [int(np.ceil(np.sqrt(N)))]*M
    else:
        k = np.random.randint(2, np.ceil(np.sqrt(N)), M)
    kmeans = partial(gen_base_partition, X, It=It)
    
    pool = mlp.Pool(mlp.cpu_count())
    
    PI = pool.map(kmeans, k)
    pool.close()
    pool.join()

    return np.transpose(PI)



def gen_cm(PI):
    """
        Generate Co-association Matrix
        
        Parameters
        ----------
        PI : ndarray of shape(n_samples, n_partitions)
            Base partitions, one column is a partition.
            
        Return
        ------
        CM : ndarray of shape(n_samples, n_samples)
            Co-association matrice.
    """
    n_samples = PI.shape[0]
    n_partitions = PI.shape[1]
    cm = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        cm[i, :] = (PI == PI[i, :]).sum(axis=1)
    return cm/n_partitions

class CluterEnsemble:
    
    def __init__(self, n_clusters=4, n_partitions=1000, max_iter=4, k_type='Fixed', cons_validation='ac', m=3, alpha=0.5):
        """
            Clustering Ensemble.
            
            Operating principle:
            ---------
                1 - generalization of several partitions using the K Means algorithm with k 
                    varies between 2 and sqrt (n_sample).

                2 - update of the co-association matrix.

                3 - generation of partitions by applying the clustering algorithm based on normalized cuts 
                    on the co_assiation matrix when the negative proof is removed.

                4 - selection of the final score with a higher degree of confidence.
            
            Parameters
            ----------
            n_cluters : int, default=4
                The number of clusters.

            n_partitions : int, default=1000
                Number of base partitions.

            max_iter : int, default=100
                Maximum number of iterations of the k-means algorithm for a single run.

            k_type : {'Fixed', 'Random'}, default='Fixed'
                The type to generate base partitions, 
            'Fixed' : k = sqrt(n_sample),
            'Random': k = random between 2 and sqrt (sample n).

            cons_validation: {'ac', 'anc', 'andc'}, default='ac'
                Type of method to use to select the final partition.
            'ac' : average Confidence of assignment of the objects to its clusters. 
            'anc': average Neighborhood Confidence of assigning the objects to its clusters.
            'andc': average dynamic neighborhood confidence of assigning the objects to its clusters.

            m : int, default=3
                Number of neighbors, used when cons_validation = 'anc'.

            alpha : float > 0, default=0.5
                Use when cons_validation = 'etc' to dynamically calculate the number of neighbors.


            Attributes
            ----------
            co_association_matrix : ndarray of shape (n_clusters, n_samples)
                Co-association matrix.

            labels_ : ndarray of shape (n_samples, )
                Labels of each point, corresponds to the best partition selected by
                one of the methods {'ac', 'anc', 'adnc'}

            partitions : ndarray of shape(n_samples, 50)
                Partitions generated by the ncut algorithm by removing negative evidence
                from the co-association matrix.
            
            quality_of_partitions : ndarray of shape (50).
                Quality measured on each partition 
                by one of the evaluation methods {'ac', 'anc', 'adnc'}
        """
        self.__n_clusters = n_clusters
        self.__n_partitions = 1000
        self.__max_iter = max_iter
        self.__k_type = k_type
        self.__cons_validation = cons_validation
        self.__m = m
        self.__alpha = alpha
        self.co_association_matrix = []
        
    def fit(self, X):
        """
            Compute clusters.
            
            Parameters
            ----------
            X : array, shape=(n_samples, n_features)
            
            Returns 
            -------
            self
                Fitted estimator.
        """
        if type(X) == pd.DataFrame:
            X = np.asarray(X)
        
        if len(self.co_association_matrix) == 0:
            PI = gen_base_partition_by_kmeans(X, It=self.__max_iter, M=self.__n_partitions, Ktype=self.__k_type)
            self.co_association_matrix = gen_cm(PI)
        self.partitions = []
        for i in np.arange(0.01, 0.51, 0.01):
            CM_ = self.co_association_matrix.copy()
            partition = ClusteringNormalizedCuts(self.__n_clusters).fit_predict(CM_)
            self.partitions.append(partition)
        self.partitions = np.transpose(self.partitions)
        
        self.quality_of_partitions = []
        if self.__cons_validation == 'ac':
            self.quality_of_partitions.append(np.apply_along_axis(lambda x: average_confidence(self.co_association_matrix,  x),
                                     0, self.partitions))
        elif self.__cons_validation == 'anc':
            self.quality_of_partitions.append(np.apply_along_axis(lambda x: average_neighborhood_confidence(self.co_association_matrix, x,
                                                                      self.__m),
                                          0, self.partitions))
        else:
            self.quality_of_partitions.append(np.apply_along_axis(lambda x: average_dynamique_neighborhood_confidence(self.co_association_matrix,
                                                                                              x, self.__alpha),
                                          0, self.partitions))
        self.quality_of_partitions = np.array(self.quality_of_partitions).ravel()
        self.labels = self.partitions[:, self.quality_of_partitions.argmax()]
        
        
    def fit_predict(self, X):
        """
            Compute clusters  and predict clustered index for each sample.
            
            Parameters
            ----------
            X : array, shape=(n_samples, n_features)
            
            Returns
            -------
            labels : array, shape [n_samples,]
                Index of the cluster each sample belongs to.
        """
        self.fit(X)
        return self.labels
    
    
    def draw_vat(self, figsize=(10, 10)):
        """
            Calculate and display the matrix for visual assessment of the cluster trend.
            
            Parameters
            ----------
            figsize : tuple, default = (10, 10)
                The dimensions of the figure.
        """
        if len(self.co_association_matrix) == 0:
            PI = gen_base_partition_by_kmeans(X, It=self.__max_iter, M=self.__n_partitions, Ktype=self.__k_type)
            self.co_association_matrix = gen_cm(PI)        
        vat_cm, index = vat(self.co_association_matrix)
        plt.figure(figsize=figsize)
        plt.title("VAT")
        im = plt.imshow(vat_cm, cmap='seismic')
        plt.colorbar(im, fraction=0.046, pad=0.04)
        plt.show()

In [3]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
categories = [
   'rec.autos', 'rec.sport.baseball','sci.crypt','sci.electronics',
     
]
print("Extracting features from the training dataset "
      "using a sparse vectorizer")

dataset = fetch_20newsgroups(subset='all', categories=categories,
                             shuffle=True, random_state=42)

print("%d documents" % len(dataset.data))
print("%d categories" % len(dataset.target_names))
print()

labels_true = dataset.target
true_k = np.unique(labels_true).shape[0]
print("Extracting features from the training dataset "
      "using a sparse vectorizer")


Extracting features from the training dataset using a sparse vectorizer
3959 documents
4 categories

Extracting features from the training dataset using a sparse vectorizer


In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer("english")
#fonction de lemmetisation et stematisation 
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# tokeniser et lemmatiser 
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result
processed_docs = []

for doc in dataset.data:
    processed_docs.append(preprocess(doc))
news = []
#retransformer le dataset en corpus de document 
for item in processed_docs:
  t = ' '.join(item)
  news.append(t)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=0.5, max_features=10000,min_df=2)
X = vectorizer.fit_transform(news)
data = X.toarray()

In [0]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
svd_2 = TruncatedSVD(200)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd_2, normalizer)

X_tranf = lsa.fit_transform(X)

In [0]:
cluster_ensemble = CluterEnsemble( n_clusters=4).fit_predict(X_tranf)

In [8]:
from sklearn import metrics
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, cluster_ensemble))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, cluster_ensemble))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, cluster_ensemble))
print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(labels_true, cluster_ensemble))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(labels_true, cluster_ensemble))


Homogeneity: 0.522
Completeness: 0.691
V-measure: 0.595
Adjusted Rand Index: 0.507
Adjusted Mutual Information: 0.595
